## Import Package

In [2]:
import numpy as np, scipy as sp,pandas as pd, matplotlib.pyplot as plt
import matplotlib, sklearn
import os,sys,csv

## Data IO

In [3]:
trainPath= '../data/train.csv'
testPath= '../test.csv'

In [5]:
def csv_nrows(fpath):
    count = 0
    with open(fpath, "r") as csvfile:
        reader = csv.reader(csvfile)
        reader.next()
        '''
        for line in reader:
            count += 1
        '''
        count = sum(1 for row in reader)
    return count

In [6]:
def csv_split(fpath, nrows=5000000):
    count = csv_nrows(fpath)
    fname, ext = os.path.splitext(fpath)
    with open(fpath, "r") as csvfile:
        reader = csv.reader(csvfile)
        header = reader.next()
        
        count = 0
        for line in reader:
            if count% nrows == 0:
                print "New Patch", int(count//nrows)
                dpath = fname + '_' + str(count//nrows) + ext
                writer = csv.writer(open(dpath, 'wb'))
                writer.writerow(header)
            writer.writerow(line)
            count += 1


    print count
    print 'Done'

In [ ]:
def csv_filter(fpath, dpath, field = 'is_booking', value='1'):
    fname, ext = os.path.splitext(fpath)
    with open(fpath, "r") as csvfile:
        reader = csv.reader(csvfile)
        header = reader.next()
        idx = header.index(field)
        
        writer = csv.writer(open(dpath, 'wb'))
        writer.writerow(header)
        count = 0
        valid = 0
        for line in reader:
            if count%1000000 == 0:
                print "Progress:", str(count), str(valid)
            if line[idx] == value:         
                writer.writerow(line)
                valid += 1
            count += 1

    print 'Done'

In [ ]:
#csv_split(trainPath)
bookingtrainPath= '../data/booking_train.csv'
csv_filter(trainPath, bookingtrainPath)

In [ ]:
print csv_nrows(trainPath)

## Feature Processing

In [4]:
headers = ['date_time', 
           'site_name',
           'posa_continent',
           'user_location_country',
           'user_location_region',
           'user_location_city',
           'orig_destination_distance',
           'user_id',
           'is_mobile',
           'is_package',
           'channel',
           'srch_ci',
           'srch_co',
           'srch_adults_cnt',
           'srch_children_cnt',
           'srch_rm_cnt',
           'srch_destination_id',
           'srch_destination_type_id',
           'hotel_continent',
           'hotel_country',
           'hotel_market',
           'is_booking',
           'cnt',
           'hotel_cluster']

new_fields = ['in_days', 'in_date']
modified = ['date_time']

updated = ['date_time',
           'in_date',
           'in_days',
           'site_name',
           'posa_continent',
           'user_location_country',
           'user_location_region',
           'user_location_city',
           'hotel_continent',
           'hotel_country',
           'hotel_market',
           'srch_destination_id',
           'orig_destination_distance',
           'user_id',
           'is_mobile',
           'is_package',
           'channel',
           'srch_adults_cnt',
           'srch_children_cnt',
           'srch_rm_cnt',
           'srch_destination_type_id',
           'is_booking',
           'cnt',
           'hotel_cluster']

In [5]:
sample = pd.read_csv('../data/train_0.csv')   

In [6]:
def time_feature_processing(df,  drop = True):
    def time_interval(row):
        if pd.isnull(row['srch_ci']) or pd.isnull(row['srch_co']):
            return 1
        st = pd.Period(row['srch_ci'],freq='D')
        et = pd.Period(row['srch_co'],freq='D') 
        return et-st
    
    def time_midpoint(row):
        if pd.isnull(row['srch_ci']) or pd.isnull(row['srch_co']):
            return 0
        st = pd.Period(row['srch_ci'],freq='D')
        et = pd.Period(row['srch_co'],freq='D') 

        date = st + (et - st) / 2
        return  date.dayofyear - 1
    
    def time_dayofyear(row):
        if pd.isnull(row['date_time']):
            return 0
        tstamp = pd.to_datetime(row['date_time'])
        #print tstamp
        return tstamp.dayofyear - 1 + tstamp.hour / 24.
    
    
    df['in_days'] = df.apply(time_interval, axis= 1)
    df['in_date'] = df.apply(time_midpoint, axis= 1)
    df['date_time'] = df.apply(time_dayofyear, axis = 1)
    
    if drop:
        df.drop('srch_ci', axis=1, inplace=True)
        df.drop('srch_co', axis=1, inplace=True)
    else:
        df['srch_ci'] = pd.PeriodIndex(df['srch_ci'], freq='D').dayofyear -1
        df['srch_co'] = pd.PeriodIndex(df['srch_co'], freq='D').dayofyear -1
        

In [7]:
def nan_feature_processing(df, colname):
    med = df[colname].median()
    df[colname] = df[colname].fillna(med)

In [8]:
part_df = sample[sample.is_booking == 1]
time_feature_processing(part_df, False)
nan_feature_processing(part_df,'orig_destination_distance')
#part_df[updated]

/home/nianzu/miniconda2/envs/work/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nianzu/miniconda2/envs/work/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nianzu/miniconda2/envs/work/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

## Feature Analysis

In [ ]:
# correlation matrix of raw feature
corr_mat = part_df[headers[:-3]].corr()
plt.matshow(corr_mat)
plt.colorbar(shrink=0.5)
plt.xticks(np.arange(len(headers)-3), headers[:-3], rotation='vertical')
plt.yticks(np.arange(len(headers)-3), headers[:-3])
plt.show()

In [ ]:
# correlation matrix of description
des = pd.read_csv('../data/destinations.csv') 

corrmat = des.corr()
plt.matshow(corrmat)
plt.colorbar(shrink=0.5)
plt.show()

## Model

### Random Forest

In [9]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import cross_val_score

import ml_metrics as mtr

In [10]:
X = part_df[updated[:-3]].as_matrix()
y = part_df.hotel_cluster.as_matrix()

In [11]:
def map_5_scorer(estimator, X, y):
    prob = estimator.predict_proba(X)
    labels = np.array(estimator.classes_)
    
    def top5(prob):
        indice = sorted(range(len(prob)), key=lambda k: prob[k], reverse=True)
        return labels[indice].tolist()
    
    y = map(lambda x:[x], y)
    y_pred = np.apply_along_axis(top5, axis=1, arr=prob)
    return mtr.mapk(y, y_pred, 5) 

In [ ]:
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="median",
                                          axis=0)),
                      ("forest", RandomForestClassifier(n_estimators=10,
                                                       n_jobs=1))])
score = cross_val_score(estimator, X, y, cv=5, scoring=map_5_scorer)
#print("Score = %.2f" % score)
print score, score.mean()

In [12]:
len(X)

401443